Complete the exercises below For **Assignment #13**.

Load the `ISLR2` and the `tidymodels` packages.

In [1]:
install.packages("ISLR2")
install.packages("tidymodels")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘shape’, ‘numDeriv’, ‘progressr’, ‘SQUAREM’, ‘diagram’, ‘lava’, ‘prodlim’, ‘warp’, ‘future.apply’, ‘iterators’, ‘listenv’, ‘parallelly’, ‘lhs’, ‘DiceDesign’, ‘sfd’, ‘sparsevctrs’, ‘patchwork’, ‘globals’, ‘clock’, ‘gower’, ‘ipred’, ‘timeDate’, ‘furrr’, ‘slider’, ‘doFuture’, ‘foreach’, ‘future’, ‘GPfit’, ‘modelenv’, ‘dials’, ‘hardhat’, ‘infer’, ‘modeldata’, ‘parsnip’, ‘recipes’, ‘rsample’, ‘tune’, ‘workflows’, ‘workflowsets’, ‘yardstick’




In [2]:
library("ISLR2")
library("tidymodels")

── Attaching packages ────────────────────────────────────── tidymodels 1.3.0 ──

✔ broom        1.0.8     ✔ recipes      1.3.1
✔ dials        1.4.1     ✔ rsample      1.3.1
✔ dplyr        1.1.4     ✔ tibble       3.3.0
✔ ggplot2      3.5.2     ✔ tidyr        1.3.1
✔ infer        1.0.9     ✔ tune         1.3.0
✔ modeldata    1.5.0     ✔ workflows    1.2.0
✔ parsnip      1.3.2     ✔ workflowsets 1.1.1
✔ purrr        1.1.0     ✔ yardstick    1.3.2

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ purrr::discard() masks scales::discard()
✖ dplyr::filter()  masks stats::filter()
✖ dplyr::lag()     masks stats::lag()
✖ recipes::step()  masks stats::step()



In this assignment we will use the `Default` dataset which includes the default status for credit card customers (`default` variable) in addition to each customer's:

1. credit card balance (`balance` variable),
1. student status (`student` variable), and,
1. income (`income` variable).

In [27]:
Default |> head()

,default,student,balance,income
,<fct>,<fct>,<dbl>,<dbl>
1,No,No,729.5265,44361.625
2,No,Yes,817.1804,12106.135
3,No,No,1073.5492,31767.139
4,No,No,529.2506,35704.494
5,No,No,785.6559,38463.496
6,No,Yes,919.5885,7491.559


We will be modeling `default` with the customer features.

Before we begin let's count how many customers fall into each `default` category.

In [28]:
Default |> count(default)

# there are 9667 customers who did not default compared to 333 who did.

default,n
<fct>,<int>
No,9667
Yes,333


The data is quite imbalanced. This will be important to keep in mind when we evaluate the performance of our model later.

Run the code below to create and training data from `Default`. We will use the "test" dataset at the end to get a final evaluation of our best model's accuracy.

In [29]:
# splitting data into test and train data sets with 90% going into the training data set
Default_split = initial_split(Default, prop = 0.90, strata = default)

Default_train = training(Default_split)
Default_test = testing(Default_split)

Create a logistic regression model called `mod`. Set the engine to `glm` and the mode to `classification`.

In [30]:
# setting the the model to be logistic regression
mod = logistic_reg(mode = "classification", engine = "glm")

Our data is imbalanced. As such, a naive model that *always* predicts a customer to **not default** would be correct quite often. Let's start by calculating the "accuracy" of a naive model. This will be the baseline accuracy by which we evaluate other models.

In [8]:
# This code calculates the accuracy of a model that always predicts default to be "No"

Default_train |>
    mutate(.pred_naive = factor('No', levels = c('No', 'Yes'))) |>
    accuracy(truth = default, .pred_naive)

# The naive model is 96.55556% accurate

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,binary,0.9655556


Let's use k-fold cross validation to evaluate the performance of a model where the outcome is `default` and the predictors are `income` and `balance`.

To start, use `vfold_cv` to generate 10 validation folds (i.e. set the `v` variable to 10). Set the `strata` argument to `default` so we preserve the distribution of `default` values in each fold.

Creat your folds below and use `glimpse` to look at the output table. Call your output folds tables "folds".

In [31]:
folds = vfold_cv(Default_train, v = 10, strata = default)

folds |> glimpse()

Rows: 10
Columns: 2
$ splits <list> [<vfold_split[8100 x 900 x 9000 x 4]>], [<vfold_split[8100 x 9…
$ id     <chr> "Fold01", "Fold02", "Fold03", "Fold04", "Fold05", "Fold06", "Fo…


The code below fits a model to each of your 10 folds. `collect_metrics` finds the average of evaluation metrics for each of your ten models.

In [32]:
mod |>
    fit_resamples(default ~ income + balance, folds) |>
    collect_metrics()

.metric,.estimator,mean,n,std_err,.config
<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
accuracy,binary,0.97388889,10,0.001520324,Preprocessor1_Model1
brier_class,binary,0.02153662,10,0.001072754,Preprocessor1_Model1
roc_auc,binary,0.94777297,10,0.003540334,Preprocessor1_Model1


❓How does the model accuracy compare to the naive model from above?

**Answer: The logistic regression model has a slightly higher accuracy at 97.39% over the naive model's 96.56% accuracy. The ROC/AUC of the logistic regression model being 0.9477 is an excellent indicator (1 is perfect) our model is better at identifing between defaulters and non compared to the naive model always assuming no default.**

Complete the cell below to evaluate a model also includes the `student` variable as as predictor.
1. use `default ~ income + balance + student` as the formula,
2. encode your `student` variable with `step_dummy`, and,
3. don't forget to `prep` your recipe!

In [33]:
rec = recipe(default ~ income + balance + student, data = Default_train) |>
  step_dummy(student) #|>
  #prep() # the function below preps for me according to the explanation of the error adding this prep line threw

mod |>
    fit_resamples(rec, folds) |>
    collect_metrics()

.metric,.estimator,mean,n,std_err,.config
<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
accuracy,binary,0.97333333,10,0.001385799,Preprocessor1_Model1
brier_class,binary,0.02144392,10,0.001070414,Preprocessor1_Model1
roc_auc,binary,0.94825489,10,0.003787814,Preprocessor1_Model1


❓Does it appear that the model that includes `student` improves upon the first model with only `income` and `balance` as predictors?

**Adding `student` to the model decreases the accuracy slightly to 97.33% versus 97.39% without `student`. This difference does not seem to be meaningful. The ROC/AUC increases slightly, but only because my code is reporting many decimal places. It is in essence unchanged. Most likely the status of `student` is not a meaningful indicator if the person will default on their loan or not.**

Finally, estimate the accuracy of an `default ~ income + balance` model on the test data, `Default_test`.

❓Does our model outperform a naive model?

**Answer: The model when applied to the test set for predictions is still performing better at 97.30% accuracy compared to the naive model at 96.56% accuracy. However, this is still a small increase overall.**

Additional note: I started to explore ROC/AUC in this assignment and assumed I would continue the comparison here. However, using the test data and got unexpected results (much lower than 0.5 which is worse than straight guessing) so I have removed that final analysis from this assignment and am continuing to explore it on my own. My initial thought is the test/train split of yes and no is not balanced, which is to be expected with so few yes entries.

In [35]:
# finding the fit using the better model and the training data
mod_fit = mod |>
  fit(default ~ income + balance, data = Default_train)

# predicting on the test set
predictions = predict(mod_fit, new_data = Default_test, type = "class") |>
  bind_cols(Default_test)

# how accurate?
metrics(predictions, truth = default, estimate = .pred_class)


.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,binary,0.97300
kap,binary,0.33208
